In [ ]:
import random
import gc
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
%matplotlib inline

sys.path.insert(0, '../../../methods')
from dnn_pavel2021 import *

In [ ]:
# random seed
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
# load data
data = np.load('../../../data\experiment_1/scenario_2/data_train.npy', allow_pickle=True)

In [ ]:
# get labels and data seperately
truths = [s['label'] for s in data]
cm = [s['cm'][np.triu_indices(7)] for s in data]

truths = np.stack(truths).squeeze(2)
cm = np.stack(cm)

In [ ]:
del data
gc.collect()

In [ ]:
# manipulate labels for classification task
res = 1
low_lim = 30
labels = np.round((truths-low_lim)*(1/res))

labels_ohe = np.zeros((labels.shape[0],121))
for i in range(labels.shape[0]):
    labels_ohe[i,labels[i].astype(int)] = 1
labels = labels_ohe

In [ ]:
# select channels
X = [np.real(cm), np.imag(cm)]
X = np.hstack(X)
y = labels

# # standardize
# for i in range(data.shape[2]):
#     for j in range(data.shape[3]):
#         for k in range(data.shape[1]):
#             data[:,k,i,j] -= np.mean(data[:,k,i,j])
#             data[:,k,i,j] /= np.std(data[:,k,i,j])
#
# # nan to zero for diagonals
# data = np.nan_to_num(data)

In [ ]:
del cm
del labels
gc.collect()

In [ ]:
# train, val split
val_ratio = 0.1
X_train, X_val, y_train, y_val, t_train, t_val = train_test_split(X, y, truths, test_size=val_ratio, random_state=seed)

In [ ]:
# dataloader
class FastTensorDataLoader:
    """
    A DataLoader-like object for a set of tensors that can be much faster than
    TensorDataset + DataLoader because dataloader grabs individual indices of
    the dataset and calls cat (slow).
    Source: https://discuss.pytorch.org/t/dataloader-much-slower-than-manual-batching/27014/6
    """
    def __init__(self, *tensors, batch_size=32, shuffle=False):
        """
        Initialize a FastTensorDataLoader.
        :param *tensors: tensors to store. Must have the same length @ dim 0.
        :param batch_size: batch size to load.
        :param shuffle: if True, shuffle the data *in-place* whenever an
            iterator is created out of this object.
        :returns: A FastTensorDataLoader.
        """
        assert all(t.shape[0] == tensors[0].shape[0] for t in tensors)
        self.tensors = tensors

        self.dataset_len = self.tensors[0].shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Calculate # batches
        n_batches, remainder = divmod(self.dataset_len, self.batch_size)
        if remainder > 0:
            n_batches += 1
        self.n_batches = n_batches
    def __iter__(self):
        if self.shuffle:
            r = torch.randperm(self.dataset_len)
            self.tensors = [t[r] for t in self.tensors]
        self.i = 0
        return self

    def __next__(self):
        if self.i >= self.dataset_len:
            raise StopIteration
        batch = tuple(t[self.i:self.i+self.batch_size] for t in self.tensors)
        self.i += self.batch_size
        return batch

    def __len__(self):
        return self.n_batches

In [ ]:
# dataloader, model, criterion and optimizer initialization
batch_size = 512
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

train_loader = FastTensorDataLoader(X_train, y_train, t_train, batch_size=batch_size, shuffle=True)
val_loader = FastTensorDataLoader(X_val, y_val, t_val, batch_size=batch_size)
model2 = Network().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params=model2.parameters(), lr=learning_rate)

In [ ]:
def get_peaks(spectrums, N):
    peaks = torch.zeros(spectrums.shape[0], N).cuda()
    for i in range(spectrums.shape[0]):
        spectrum = spectrums[i,:]
        spectrum_temp = torch.cat([torch.tensor([-1000]).cuda(),spectrum,torch.tensor([-1000]).cuda()])
        crossings = ((torch.diff(torch.sign(torch.diff(spectrum_temp))) == -2)*1)
        crossings = torch.where(crossings==1)[0]
        crossing_spectrum = spectrum[crossings]
        crossings_sub = torch.argsort(crossing_spectrum, descending=True, dim=0)
        peaks[i,:] = crossings[crossings_sub][:N]

    return peaks

In [ ]:
del X_train
del X_val
del y_train
del y_val
gc.collect()

In [ ]:
# training
num_epochs = 558
best_loss = 1000
lr_decay_patience = 10
lr_decay_counter = 0
train_loss_list = np.zeros(num_epochs)
train_rmse_list = np.zeros(num_epochs)
train_acc_list = np.zeros(num_epochs)
val_loss_list = np.zeros(num_epochs)
val_rmse_list = np.zeros(num_epochs)
val_acc_list = np.zeros(num_epochs)
# train_loss_list = np.load('/content/drive/MyDrive/Research/Experiments/train_loss.npy')
# train_rmse_list = np.load('/content/drive/MyDrive/Research/Experiments/train_rmse.npy')
# train_acc_list = np.load('/content/drive/MyDrive/Research/Experiments/train_acc.npy')
# val_loss_list = np.load('/content/drive/MyDrive/Research/Experiments/val_loss.npy')
# val_rmse_list = np.load('/content/drive/MyDrive/Research/Experiments/val_rmse.npy')
# val_acc_list = np.load('/content/drive/MyDrive/Research/Experiments/val_acc.npy')

for epoch in range(0,num_epochs):
    model2.train()
    running_loss = 0
    counter = 0

    for batch_id, (X, y, t) in tqdm(enumerate(train_loader), leave=False, desc="training"):
        X = torch.tensor(X).float().to(device)
        y = torch.tensor(y).float().to(device)
        t = torch.tensor(t).float().to(device)

        prob = model2(X)
        loss = criterion(prob,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.detach().data * y.shape[0]
        counter += y.shape[0]

        del X
        del y
        del loss
        del prob
    gc.collect()


    train_loss = running_loss/counter
    train_loss_list[epoch] = train_loss

    torch.cuda.empty_cache()

    # validation
    with torch.no_grad():
        model2.eval()
        running_loss = 0
        counter = 0

        for batch_id, (X, y, t) in tqdm(enumerate(val_loader), leave=False, desc="validation"):
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).float().to(device)
            t = torch.tensor(t).float().to(device)

            prob = model2(X)
            loss = criterion(prob,y)

            running_loss += loss.detach().data * y.shape[0]
            counter += y.shape[0]

            del X
            del y
            del loss
            del prob
        gc.collect()

        val_loss = running_loss/counter
        val_loss_list[epoch] = val_loss

    print(f"epoch {epoch+1}, train-loss {train_loss:.4f}, val-loss {val_loss:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        lr_decay_counter = 0
        torch.save(model2.state_dict(), '../../../results\experiment_1/scenario_2/dnn_pavel2021.pt')
        print(f"model is saved")
    else:
        lr_decay_counter += 1

    if lr_decay_counter == lr_decay_patience:
        lr_decay_counter = 0
        learning_rate *= 0.7
        learning_rate = max([learning_rate, 1e-7])
        print(f"learning rate decayed to {learning_rate:.4f}")

        for g in optimizer.param_groups:
            g['lr'] = learning_rate

    np.save('../../../results\experiment_1/scenario_2/train_loss.npy', train_loss_list)
    np.save('../../../results\experiment_1/scenario_2/train_rmse.npy', train_rmse_list)
    np.save('../../../results\experiment_1/scenario_2/train_acc.npy', train_acc_list)
    np.save('../../../results\experiment_1/scenario_2/val_loss.npy', val_loss_list)
    np.save('../../../results\experiment_1/scenario_2/val_rmse.npy', val_rmse_list)
    np.save('../../../results\experiment_1/scenario_2/val_acc.npy', val_acc_list)


In [ ]:
# loss curves
plt.figure(figsize=(10,5))
plt.plot(range(len(train_loss_list))[:558], train_loss_list[:558])
plt.plot(range(len(val_loss_list))[:558], val_loss_list[:558])
plt.title("training and validation loss curves")
plt.xlabel("epoch")
plt.ylabel("cross entropy loss")
plt.legend(["train", "val"])
plt.grid()

In [ ]:
snr_list = [-20,-15,-10,-5,0,5,10,15,20,25,30]
T_list = [100,200,500,1000,2000,5000,10000]
rmse_list1 = []
rmse_list2 = []

for snr in snr_list:
    # load data
    data = np.load(f'../../../data\experiment_1/scenario_2/data_test_snr{snr}_t1000.npy', allow_pickle=True)

    # get labels and data seperately
    cm = [s['cm'][np.triu_indices(7)] for s in data]
    truths = [s['label'] for s in data]

    cm = np.stack(cm)
    truths = np.stack(truths).squeeze(2)

    # manipulate labels for classification task
    res = 1
    low_lim = 30
    labels = np.round((truths-low_lim)*(1/res))

    labels_ohe = np.zeros((labels.shape[0],121))
    for i in range(labels.shape[0]):
        labels_ohe[i,labels[i].astype(int)] = 1
    labels = labels_ohe

    # select channels
    X = [np.real(cm), np.imag(cm)]
    X = np.hstack(X)
    y = labels

    # # standardize
    # for i in range(data.shape[2]):
    #     for j in range(data.shape[3]):
    #         for k in range(data.shape[1]):
    #             data[:,k,i,j] -= np.mean(data[:,k,i,j])
    #             data[:,k,i,j] /= np.std(data[:,k,i,j])
    #
    # # nan to zero for diagonals
    # data = np.nan_to_num(data)

    # dataloader, model, criterion and optimizer initialization
    batch_size = 512
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    test_loader = FastTensorDataLoader(X, y, truths, batch_size=batch_size)
    model2 = Network().to(device)
    model2.load_state_dict(torch.load('../../../results\experiment_1/scenario_2/dnn_pavel2021.pt', map_location=torch.device('cpu')))
    criterion = nn.BCEWithLogitsLoss()

    # testing
    with torch.no_grad():
        model2.eval()
        running_loss = 0
        counter = 0
        p_list = []
        y_list = []
        t_list = []

        for batch_id, (X, y, t) in enumerate(test_loader):
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).float().to(device)
            t = torch.tensor(t).float().to(device)

            prob = model2(X)
            p = get_peaks(prob,4)
            loss = criterion(prob,y)

            if snr==10 and batch_id==0:
                np.save(f'../../../results\experiment_1/scenario_2/dnn_pavel2021_spectrum_{t[0].cpu().numpy()}deg.npy', F.softmax(prob[0,:], dim=0).cpu().numpy())

            p = (p-low_lim)/res

            t, _ = torch.sort(t)
            p, _ = torch.sort(p)

            running_loss += loss.item() * y.shape[0]
            counter += y.shape[0]
            p_list.append(p.detach().data)
            y_list.append(y.detach().data)
            t_list.append(t.detach().data)

            del X
            del y
            del p
            del loss
            del prob
        gc.collect()

        p_list = torch.cat(p_list,dim=0)
        y_list = torch.cat(y_list,dim=0)
        t_list = torch.cat(t_list,dim=0)
        y_list = y_list.nonzero()[:,1].reshape(-1,4)
        test_loss = running_loss/counter
        test_acc = torch.sum(p_list.reshape(-1,1)==y_list.reshape(-1,1)) / y_list.reshape(-1,1).shape[0]
        test_rmse = torch.sqrt(torch.mean((t_list-(p_list*res+low_lim))**2))
        rmse_list1.append(test_rmse.item())

        del y_list
        gc.collect()

    print(f"snr {snr}dB, T {1000}, test-loss {test_loss:.4f}, test-rmse {test_rmse:.4f}, test-acc {test_acc:.4f}")

    np.save(f'../../../results\experiment_1/scenario_2/dnn_pavel2021_preds_snr{snr}_t1000.npy', (p_list*res+low_lim).cpu().numpy())
    np.save(f'../../../results\experiment_1/scenario_2/dnn_pavel2021_truths_snr{snr}_t1000.npy', t_list.cpu().numpy())

for T in T_list:
    # load data
    data = np.load(f'../../../data\experiment_1/scenario_2/data_test_snr-10_t{T}.npy', allow_pickle=True)

    # get labels and data seperately
    cm = [s['cm'][np.triu_indices(7)] for s in data]
    truths = [s['label'] for s in data]

    cm = np.stack(cm)
    truths = np.stack(truths).squeeze(2)

    # manipulate labels for classification task
    res = 1
    low_lim = 30
    labels = np.round((truths-low_lim)*(1/res))

    labels_ohe = np.zeros((labels.shape[0],121))
    for i in range(labels.shape[0]):
        labels_ohe[i,labels[i].astype(int)] = 1
    labels = labels_ohe

    # select channels
    X = [np.real(cm), np.imag(cm)]
    X = np.hstack(X)
    y = labels

    # # standardize
    # for i in range(data.shape[2]):
    #     for j in range(data.shape[3]):
    #         for k in range(data.shape[1]):
    #             data[:,k,i,j] -= np.mean(data[:,k,i,j])
    #             data[:,k,i,j] /= np.std(data[:,k,i,j])
    #
    # # nan to zero for diagonals
    # data = np.nan_to_num(data)

    # dataloader, model, criterion and optimizer initialization
    batch_size = 512
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    test_loader = FastTensorDataLoader(X, y, truths, batch_size=batch_size)
    model2 = Network().to(device)
    model2.load_state_dict(torch.load('../../../results\experiment_1/scenario_2/dnn_pavel2021.pt', map_location=torch.device('cpu')))
    criterion = nn.BCEWithLogitsLoss()

    # testing
    with torch.no_grad():
        model2.eval()
        running_loss = 0
        counter = 0
        p_list = []
        y_list = []
        t_list = []

        for batch_id, (X, y, t) in enumerate(test_loader):
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).float().to(device)
            t = torch.tensor(t).float().to(device)

            prob = model2(X)
            p = get_peaks(prob,4)
            loss = criterion(prob,y)

            p = (p-low_lim)/res

            t, _ = torch.sort(t)
            p, _ = torch.sort(p)

            running_loss += loss.item() * y.shape[0]
            counter += y.shape[0]
            p_list.append(p.detach().data)
            y_list.append(y.detach().data)
            t_list.append(t.detach().data)

            del X
            del y
            del p
            del loss
            del prob
        gc.collect()

        p_list = torch.cat(p_list,dim=0)
        y_list = torch.cat(y_list,dim=0)
        t_list = torch.cat(t_list,dim=0)
        y_list = y_list.nonzero()[:,1].reshape(-1,4)
        test_loss = running_loss/counter
        test_acc = torch.sum(p_list.reshape(-1,1)==y_list.reshape(-1,1)) / y_list.reshape(-1,1).shape[0]
        test_rmse = torch.sqrt(torch.mean((t_list-(p_list*res+low_lim))**2))
        rmse_list2.append(test_rmse.item())

        del y_list
        gc.collect()

    print(f"snr -10dB, T {T}, test-loss {test_loss:.4f}, test-rmse {test_rmse:.4f}, test-acc {test_acc:.4f}")

    np.save(f'../../../results\experiment_1/scenario_2/dnn_pavel2021_preds_snr-10_t{T}.npy', (p_list*res+low_lim).cpu().numpy())
    np.save(f'../../../results\experiment_1/scenario_2/dnn_pavel2021_truths_snr-10_t{T}.npy', t_list.cpu().numpy())

np.save('../../../results\experiment_1/scenario_2/dnn_pavel2021_rmse1.npy', rmse_list1)
np.save('../../../results\experiment_1/scenario_2/dnn_pavel2021_rmse2.npy', rmse_list2)

In [ ]:
# plot rmse values
plt.figure()
plt.plot(snr_list, rmse_list1, '-o')
plt.title("rmse values for different snr levels")
plt.xlabel("snr (dB)")
plt.ylabel("rmse (deg)")
plt.legend(['DNN Pavel 2021'])
plt.yscale("log")
plt.grid()

plt.figure()
plt.plot(T_list, rmse_list2, '-o')
plt.title("rmse values for different snapshot numbers")
plt.xlabel("T")
plt.ylabel("rmse (deg)")
plt.legend(['DNN Pavel 2021'])
plt.yscale("log")
plt.grid()